***GENERATED CODE FOR meteranomaly PIPELINE***
***DON'T EDIT THIS CODE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BADGE_NBR', 'transformation_label': 'String Indexer'}], 'feature': 'BADGE_NBR', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '88698', 'mean': '', 'stddev': '', 'min': 'M00006144', 'max': 'M02590464', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'BADGE_NBR', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BADGE_NBR')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'WEEK_DAY_IND', 'transformation_label': 'String Indexer'}], 'feature': 'WEEK_DAY_IND', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '88698', 'mean': '', 'stddev': '', 'min': 'N', 'max': 'Y', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'WEEK_DAY_IND', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('WEEK_DAY_IND')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MONTH_NM', 'transformation_label': 'String Indexer'}], 'feature': 'MONTH_NM', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '88698', 'mean': '', 'stddev': '', 'min': 'FEBRUARY ', 'max': 'MARCH    ', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'MONTH_NM', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MONTH_NM')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CITY', 'transformation_label': 'String Indexer'}], 'feature': 'CITY', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '88698', 'mean': '', 'stddev': '', 'min': 'AGUILA', 'max': 'YUMA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'CITY', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CITY')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MFG_CD', 'transformation_label': 'String Indexer'}], 'feature': 'MFG_CD', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '88698', 'mean': '', 'stddev': '', 'min': 'ELSTER', 'max': 'LG', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'MFG_CD', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MFG_CD')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SP_TYPE_CD', 'transformation_label': 'String Indexer'}], 'feature': 'SP_TYPE_CD', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '88698', 'mean': '', 'stddev': '', 'min': 'COMM', 'max': 'RES', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'SP_TYPE_CD', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SP_TYPE_CD')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

%run meteranomalyHooks.ipynb
try:
	sourcePreExecutionHook()

	meteranomalydbfs = DBFSConnector.fetch(spark, "{'url': '/Demo/MeterAnomalyTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

	sourcePostExecutionHook(meteranomalydbfs)

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
%run meteranomalyHooks.ipynb
try:
	transformationPreExecutionHook()

	meteranomalyautofe = TransformationMain.run(meteranomalydbfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "BADGE_NBR", "transformation_label": "String Indexer"}], "feature": "BADGE_NBR", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "88698", "mean": "", "stddev": "", "min": "M00006144", "max": "M02590464", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INT_SUM_DAY_USG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "88698", "mean": "156.96", "stddev": "81.54", "min": "0.0", "max": "983.62", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MONTH_AVG_USG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "88698", "mean": "190.74", "stddev": "113.02", "min": "0.0", "max": "979.46", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "HIGHEST_INT_DAY_USG", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "88698", "mean": "11.12", "stddev": "4.98", "min": "0.0", "max": "46.33", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "METER_TYPE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "88698", "mean": "60.0", "stddev": "0.0", "min": "60", "max": "60", "missing": "0"}}, {"transformationsData": [{"feature_label": "WEEK_DAY_IND", "transformation_label": "String Indexer"}], "feature": "WEEK_DAY_IND", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "88698", "mean": "", "stddev": "", "min": "N", "max": "Y", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "MONTH_NM", "transformation_label": "String Indexer"}], "feature": "MONTH_NM", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "88698", "mean": "", "stddev": "", "min": "FEBRUARY ", "max": "MARCH    ", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DAILY_AVG_TEMP", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "88698", "mean": "58.1", "stddev": "8.43", "min": "40.5", "max": "74.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "CITY", "transformation_label": "String Indexer"}], "feature": "CITY", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "88698", "mean": "", "stddev": "", "min": "AGUILA", "max": "YUMA", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "MFG_CD", "transformation_label": "String Indexer"}], "feature": "MFG_CD", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "88698", "mean": "", "stddev": "", "min": "ELSTER", "max": "LG", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "SP_TYPE_CD", "transformation_label": "String Indexer"}], "feature": "SP_TYPE_CD", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "88698", "mean": "", "stddev": "", "min": "COMM", "max": "RES", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "USAGE_VAR_PER", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "88698", "mean": "-355.89", "stddev": "19155.55", "min": "-2617800.0", "max": "94.73", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ISSUE_IND", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "88698", "mean": "0.36", "stddev": "0.48", "min": "0", "max": "1", "missing": "0"}}, {"feature": "BADGE_NBR_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "88698", "mean": "882.47", "stddev": "654.68", "min": "0.0", "max": "2234.0", "missing": "0"}}, {"feature": "WEEK_DAY_IND_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "88698", "mean": "0.29", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}}, {"feature": "MONTH_NM_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "88698", "mean": "0.86", "stddev": "0.8", "min": "0", "max": "2", "missing": "0"}}, {"feature": "CITY_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "88698", "mean": "8.85", "stddev": "14.22", "min": "0.0", "max": "92.0", "missing": "0"}}, {"feature": "MFG_CD_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "88698", "mean": "0.07", "stddev": "0.25", "min": "0", "max": "1", "missing": "0"}}, {"feature": "SP_TYPE_CD_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "88698", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "3", "missing": "0"}}]}))

	transformationPostExecutionHook(meteranomalyautofe)

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
%run meteranomalyHooks.ipynb
try:
	mlPreExecutionHook()

	dataAutoML=functionClassification(meteranomalyautofe, ["INT_SUM_DAY_USG", "MONTH_AVG_USG", "HIGHEST_INT_DAY_USG", "METER_TYPE", "DAILY_AVG_TEMP", "USAGE_VAR_PER", "BADGE_NBR_stringindexer", "WEEK_DAY_IND_stringindexer", "MONTH_NM_stringindexer", "CITY_stringindexer", "MFG_CD_stringindexer", "SP_TYPE_CD_stringindexer"], "ISSUE_IND")

	mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

